<a href="https://colab.research.google.com/github/paryoja/deeplearningstudy/blob/master/Using_Tensorboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensorboard를 써보자

Tensorboard는 tensorflow의 그래프를 쉽게 보고,

중간의 학습결과의 변화를 쉽게 알 수 있게 보여주는 툴입니다.

개인 컴퓨터에 tensorflow를 설치하면 쉽게 사용할 수 있지만,

colab 환경에서는 사용하기 쉽지 않습니다.


다음의 코드를 수행하면 tensorboard를 ./log 디렉토리를 기준으로 실행됩니다.

출처: https://stackoverflow.com/questions/47818822/can-i-use-tensorboard-with-google-colab





In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

LOG_DIR = './log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

get_ipython().system_raw('./ngrok http 6006 &')

! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

위에 나오는 링크를 누르면 colab 환경에서의 텐서보드를 접속할 수 있습니다.



당장은
```
No dashboards are active for the current data set.
Probable causes:

You haven’t written any data to your event files.
TensorBoard can’t find your event files.
If you’re new to using TensorBoard, and want to find out how to add data and set up your event files, check out the README and perhaps the TensorBoard tutorial.
If you think TensorBoard is configured properly, please see the section of the README devoted to missing data problems and consider filing an issue on GitHub.

Last reload: Mon Oct 29 2018 22:18:50 GMT+0900 (한국 표준시)
Data location: ./log
```
와 같은 메시지만 뜰 것입니다.

간단한 그래프를 만들어 보고 그것을 보겠습니다.

In [0]:
import tensorflow as tf

# variable 
x = tf.Variable(0, dtype=tf.float32, name="my_variable")

# constant
a = tf.constant(2, dtype=tf.float32, name="first_const")
b = tf.constant(1.5, dtype=tf.float32, name="second_const")

# 연산
c = tf.add(x, a, name="my_add")
d = tf.multiply(c, b, name="my_mul")

# assign 연산
my_assign = tf.assign(x, d, name="assign_to_my_variable")

# 세션을 하나 만들고
sess = tf.Session()

# 그 세션의 graph를 ./log/simple_graph 밑에 저장합니다
summary_writer = tf.summary.FileWriter('./log/simple_graph', sess.graph)

# 결과 보기

![그림](https://i.imgur.com/CoSwB0El.png) 


과 같은 결과나 나왔나요?



몇몇분은 저런 모양이 두개나 세개일수도 있습니다.

그것은 위 셀을 몇번이나 호출했는지에 따라 달라집니다.


보시다시비 텐서플로우의 그래프는 계속 누적이 되므로

처음부터 다시 시작하고 싶을 수 있습니다.



그 경우 메뉴의 런타임 -> 런타임 다시 시작을 눌러주시고 

그래프 그리는 부분을 다시 실행하고,

텐서보드의 새로고침을 누르면 딱 하나만 그려져 있을 겁니다.

(텐서보드를 실행시키는 코드 - 맨처음 코드 - 는  다시 수행하지 않아도 됩니다)




그림을 보면 제가 짠 코드를 잘 보여주고 있습니다.

my_variable 이라는 것과 first_const를 더하고 그걸 my_add라 했으며,

이에 second_const를 곱하여 my_mul이라고 이름 붙였습니다.

그리고 assign_to_my_variable 도 보이네요. (비록 이름이 잘리긴 했지만 ㅠ)

# 값의 변화 보기

텐서보드의 용도는 제가 구현한 그래프를 확인하는데 그치지 않습니다.

다음과 같이 세션마다 변화하는 값을 추적하기에도 좋습니다.

x 의 값은 0으로 초기화 되고, 3 -> 7.5 -> ... 식으로 증가하는 수열을 이룹니다.

In [0]:
# 여기서 x는 scalar 값입니다.

tf.summary.scalar('watch_my_variable', x)

# 앞서 정의한 summary를 모두 모으는 역할을 합니다.
merged = tf.summary.merge_all()

# 변수를 사용하니 초기화를 하고
sess.run(tf.global_variables_initializer())

for i in range(100):
  # 100번을 돌면서 값을 기록합니다.
  summary, _ = sess.run([merged, my_assign])
  summary_writer.add_summary(summary, i)

![대체 텍스트](https://i.imgur.com/APDrkjP.png)

위에 탭을 보면 scalars 라고 있습니다.

이를 누르면 다음과 같이 그래프가 그려져 있습니다.

나중에는 너무 켜져서 보이지 않는데 그래프 영역을 움직이면 보실 수 있습니다.




# 히스토그램


텐서플로우에서는 일반적으로 텐서를 다룹니다.


그렇다면 tf.summary.scalar와 같이 하나의 값의 변화를 보기보다는

전체적인 값의 분포의 변화를 보고 싶을 수 있습니다.

그 경우에는 histogram을 사용할 수 있습니다.



In [0]:
import tensorflow as tf

# 지금 그려져 있는 그래프를 초기화 합니다.
tf.reset_default_graph()

# x는 100, 100 행렬입니다.
x = tf.Variable(tf.random_normal([100, 100]), dtype=tf.float32, name="my_variable")
a = tf.constant(2, dtype=tf.float32, name="first_const") 

# (100, 100) 에 () 의 scalar를 더하므로,
# broadcasting에 의해서 scalar가 (100, 100)으로 뻥튀기 된 후 더해집니다.
c = tf.add(x, a, name="my_add")

my_assign = tf.assign(x, c, name="assign_to_my_variable")

sess = tf.Session()

summary_writer = tf.summary.FileWriter('./log/simple_graph_0', sess.graph)

tf.summary.histogram('watch_my_variable', x)
merged = tf.summary.merge_all()

sess.run(tf.global_variables_initializer())

for i in range(100):
  summary, _ = sess.run([merged, my_assign])
  summary_writer.add_summary(summary, i)

![대체 텍스트](https://i.imgur.com/1UvFIgp.png)


와 같은 그래프를 보실 수 있습니다.


히스토그램은 중학교때 배운 도수분포표입니다.


국어 점수 60~70 사이 5명, 70~80 사이 7명, ... 이런 데이터를 그림으로 나타낸 것입니다.

다만 여기서는 점수 구간을 알고리즘에 의해 나누고,

데이터 사용량을 줄이기 위해 sampling을 수행해서 그림이 정확하지는 않습니다.


하지만 값의 분포의 변화를 보기에는 나쁘지 않습니다.


https://www.tensorflow.org/guide/summaries_and_tensorboard

에서 좀 더 자세한 내용을 확인하세요!


